In [21]:
# ! pip install surprise

In [1]:
import re
import json
import surprise
import pandas as pd
from tqdm import tqdm

In [16]:
# ! ls ./data

In [23]:
import gc
gc.collect()

90

In [22]:
user_item = []
with open("./data/australian_users_items.json") as f:
    s = "[" + f.read().strip() + "]"
    s = s.replace("\\", ' ')
    s = s.replace('"Glow Ball"', 'Glow Ball')
    s = s.replace('"New life"', "New life")
    double = set(re.findall(r'"[^"]+"', s))
    for x in tqdm(double):
        tmp = x.replace("'", " ")
        tmp = tmp.replace('"', "'")
        s = s.replace(x, tmp)
    s = s.replace("\n", ",\n")
    s = s.replace("'", '"')
    s = s.replace('"One pioneer  s story"', 'One pioneer  s story')
    user_item = json.loads(s)

In [15]:
with open("./data/fixed_australian_users_items.json", 'w') as f:
    json.dump(user_item, f)

In [17]:
with open("./data/fixed_australian_users_items.json") as f:
    user_item = json.loads(f.read())

In [20]:
# user_item[-1]